In [15]:
#Python Basics
from IPython.display import clear_output
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
import string
import pickle
import requests
from bs4 import BeautifulSoup
import lyricsgenius
from collections import Counter
from itertools import chain
import pprint

#NLP & ML
import editdistance as ed
from sklearn.pipeline import Pipeline
import gensim
import nltk
from nltk import pos_tag, pos_tag_sents
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer

clear_output(wait=True)

In [2]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
eng_stopwords = stopwords.words('english') 
clear_output(wait=True)

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1108)>
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1108)>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify
[nltk_data]     failed: unable to get local issuer certificate
[nltk_data]     (_ssl.c:1108)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1108)>


In [3]:
kanye = pd.read_pickle("kanye_lyrics.pkl")
kanye_jik = pd.read_pickle('kanye_jik.pkl')
kanye = pd.concat([kanye, kanye_jik])
kanye.head()

,albums,track_number,track,writers,track_length,Year,old_kanye,clean_track,features,lyrics
0,The College Dropout,1,Intro (Skit),Kanye West,0:19,2004,1,Intro (Skit),None,"\n\n[Intro: DeRay Davis]\nKanye, can I talk to..."
1,The College Dropout,2,We Don't Care,West Miri Ben-Ari Ross Vannelli,3:59,2004,1,We Don't Care,None,\n\n[Intro]\nOh yeah...\nI got the perfect son...
2,The College Dropout,3,Graduation Day,West John Stephens Ben-Ari,1:22,2004,1,Graduation Day,None,"[Spoken: DeRay]\nWhat in the fuck was that, Ka..."
3,The College Dropout,4,All Falls Down (featuring Syleena Johnson),West Lauryn Hill,3:43,2004,1,All Falls Down,Syleena_Johnson,"[Chorus: Syleena Johnson & Kanye West]\nOh, wh..."
4,The College Dropout,5,I'll Fly Away,Albert E. Brumley,1:09,2004,1,I'll Fly Away,None,\n\nOne glad morning\nWhen this life is over\n...


In [4]:
albums = list(kanye['albums'].unique())
albums


['The College Dropout',
 'Late Registration',
 'Graduation',
 '808s & Heartbreak',
 'My Beautiful Dark Twisted Fantasy',
 'Cruel Summer',
 'Yeezus',
 'The Life of Pablo',
 'Ye',
 'Singles',
 'Cruel Winter',
 'Jesus is King']

# Feature Generation

Let's look at indirect/direct features for Kanye overall and individual albums

# Indirect Features
Some more experimental features.

 - count of sentences

 - count of words

 - count of unique words

 - count of letters

 - count of punctuations

 - count of uppercase words/letters

 - count of stop words

 - avg length of each word

In [5]:
## Indirect features

#Sentense count in each comment:
    #  '\n' can be used to count the number of sentences in each comment
kanye['count_sent']=kanye["lyrics"].apply(lambda x: len(re.findall("\n",str(x)))+1)
#Word count in each comment:
kanye['count_word']=kanye["lyrics"].apply(lambda x: len(str(x).split()))
#Unique word count
kanye['count_unique_word']=kanye["lyrics"].apply(lambda x: len(set(str(x).split())))
#Letter count
kanye['count_letters']=kanye["lyrics"].apply(lambda x: len(str(x)))
#punctuation count
kanye["count_punctuations"] =kanye["lyrics"].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
#upper case words count
kanye["count_words_upper"] = kanye["lyrics"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
#Number of stopwords
kanye["count_stopwords"] = kanye["lyrics"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
#Average length of the words
kanye["mean_word_len"] = kanye["lyrics"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

#derived features
#Word count percent in each song:
kanye['word_unique_percent'] = kanye['count_unique_word']*100/kanye['count_word']
#Punct percent in each song:
kanye['punct_percent']=kanye['count_punctuations']*100/kanye['count_word']

In [6]:
#derived features
#Word count percent in each song:
kanye['word_unique_percent'] = kanye['count_unique_word']*100/kanye['count_word']
#derived features
#Punct percent in each song:
kanye['punct_percent']=kanye['count_punctuations']*100/kanye['count_word']

In [7]:
kanye[['old_kanye', 'count_sent', 'count_word', 'count_unique_word', 
       'count_letters', 'count_punctuations', 'count_words_upper',
       'count_stopwords', 'mean_word_len', 'word_unique_percent', 'punct_percent']].groupby(['old_kanye'], as_index=False).mean()

,old_kanye,count_sent,count_word,count_unique_word,count_letters,count_punctuations,count_words_upper,count_stopwords,mean_word_len,word_unique_percent,punct_percent
0,0,71.981132,452.433962,208.754717,2338.433962,120.603774,19.094340,183.924528,4.158140,47.206680,26.78931
1,1,82.892857,579.285714,254.083333,2978.321429,148.011905,24.154762,239.416667,4.243931,47.280344,27.64002


In [19]:
kanye[['old_kanye', 'count_word', 'count_unique_word', 'count_stopwords']].groupby(['old_kanye'], as_index=False).mean()

,old_kanye,count_word,count_unique_word,count_stopwords
0,0,452.433962,208.754717,183.924528
1,1,579.285714,254.083333,239.416667


In [9]:
kanye[~kanye['albums'].isin(['Cruel Winter', 'Cruel Summer'])].groupby(
    'albums')['count_sent'].mean().sort_values(ascending = False)


albums
My Beautiful Dark Twisted Fantasy    95.692308
Graduation                           91.692308
The College Dropout                  83.904762
Yeezus                               83.714286
The Life of Pablo                    76.850000
Singles                              73.142857
Late Registration                    70.238095
Ye                                   67.000000
808s & Heartbreak                    61.166667
Jesus is King                        49.181818
Name: count_sent, dtype: float64

In [10]:
kanye[~kanye['albums'].isin(['Cruel Winter', 'Cruel Summer'])].groupby(
    'albums')['count_word'].mean().sort_values(ascending = False)


albums
The College Dropout                  669.761905
My Beautiful Dark Twisted Fantasy    661.076923
Graduation                           562.692308
Late Registration                    508.761905
Yeezus                               497.714286
The Life of Pablo                    486.400000
Singles                              464.857143
Ye                                   459.571429
808s & Heartbreak                    361.000000
Jesus is King                        291.272727
Name: count_word, dtype: float64

In [11]:
kanye[~kanye['albums'].isin(['Cruel Winter', 'Cruel Summer'])].groupby(
    'albums')['count_unique_word'].mean().sort_values(ascending = False)


albums
The College Dropout                  295.619048
My Beautiful Dark Twisted Fantasy    289.538462
Graduation                           238.076923
Late Registration                    238.000000
Yeezus                               237.428571
The Life of Pablo                    223.500000
Ye                                   220.000000
Singles                              205.857143
Jesus is King                        139.545455
808s & Heartbreak                    134.916667
Name: count_unique_word, dtype: float64

In [12]:
kanye[~kanye['albums'].isin(['Cruel Winter', 'Cruel Summer'])].groupby(
    'albums')['word_unique_percent'].mean().sort_values(ascending = False)

albums
The College Dropout                  51.912524
Late Registration                    49.922418
My Beautiful Dark Twisted Fantasy    49.162116
Jesus is King                        48.747641
The Life of Pablo                    48.255908
Ye                                   47.752725
Yeezus                               47.406309
Graduation                           43.357206
Singles                              42.343336
808s & Heartbreak                    37.111512
Name: word_unique_percent, dtype: float64

In [13]:
from scipy.stats import ttest_ind

def success_vs_fail_report(feature, try_log = False):
    print (feature)
    
    success_values = kanye[feature][kanye['old_kanye'] == 1]
    print ("SUCCESS", "Mean:", "{:.3f}".format(success_values.mean()), "StdDev:", "{:.3f}".format(success_values.std()))
    
    fail_values = kanye[feature][kanye['old_kanye'] == 0]
    print ("FAIL   ", "Mean:", "{:.3f}".format(fail_values.mean()), "StdDev:", "{:.3f}".format(fail_values.std()))
    
    ttest = ttest_ind(success_values, fail_values, equal_var=False)
    print ("\nTStat:", "{:.3f}".format(ttest.statistic))
    print ("P-Value", "{:.2f}\n".format(ttest.pvalue))
    if try_log:
        ttest_of_log = ttest_ind(np.log(success_values + 1), np.log(fail_values + 1), equal_var=False)
        print ("TStat (log):", "{:.3f}".format(ttest_of_log.statistic))

In [18]:
indirect_features = ['count_sent', 'count_word', 'count_unique_word', 
       'count_letters', 'count_punctuations', 'count_words_upper',
       'count_stopwords', 'mean_word_len', 'word_unique_percent', 'punct_percent']
[success_vs_fail_report(i) for i in indirect_features]

count_sent
SUCCESS Mean: 82.893 StdDev: 41.096
FAIL    Mean: 71.981 StdDev: 32.459

TStat: 1.726
P-Value 0.09

count_word
SUCCESS Mean: 579.286 StdDev: 389.445
FAIL    Mean: 452.434 StdDev: 230.830

TStat: 2.393
P-Value 0.02

count_unique_word
SUCCESS Mean: 254.083 StdDev: 151.252
FAIL    Mean: 208.755 StdDev: 108.480

TStat: 2.039
P-Value 0.04

count_letters
SUCCESS Mean: 2978.321 StdDev: 2007.068
FAIL    Mean: 2338.434 StdDev: 1212.810

TStat: 2.326
P-Value 0.02

count_punctuations
SUCCESS Mean: 148.012 StdDev: 118.200
FAIL    Mean: 120.604 StdDev: 61.775

TStat: 1.775
P-Value 0.08

count_words_upper
SUCCESS Mean: 24.155 StdDev: 23.227
FAIL    Mean: 19.094 StdDev: 14.314

TStat: 1.578
P-Value 0.12

count_stopwords
SUCCESS Mean: 239.417 StdDev: 171.641
FAIL    Mean: 183.925 StdDev: 98.494

TStat: 2.402
P-Value 0.02

mean_word_len
SUCCESS Mean: 4.244 StdDev: 1.098
FAIL    Mean: 4.158 StdDev: 0.242

TStat: 0.690
P-Value 0.49

word_unique_percent
SUCCESS Mean: 47.280 StdDev: 12.337
FAIL 

[None, None, None, None, None, None, None, None, None, None]

Let's also look at some of the most common words and phrases Kanye uses. Minus the obscenity

In [16]:
#For each album, what are some of the top bi, tri, and quadgrams as Kanye's hallmark phrases
def find_ngrams(input_list, n):
    return list(zip(*[input_list[i:] for i in range(n)]))


def albumify(df, n):
    pp = pprint.PrettyPrinter(indent=4)
    
    for i in list(df['albums'].unique()):
        album_lyrics = df[df['albums'] == i]['lyrics'].str.lower().apply(
            lambda x: re.sub(r'intro', r'', x)).apply(
            lambda x: re.sub(r'\d+', r' ', x)).apply(
            lambda y: re.sub(r'\W+', r' ',y)).apply(
            lambda z: re.sub(r"_+",r" ", z)).apply(
            lambda y: re.sub(r'verse', r'', y)).apply(
            lambda z: re.sub(r'chorus', r'', z)).apply(
            lambda a: re.sub(r'produce', r'', a)).apply(
            lambda x: re.sub("[\[].*?[\]]", "", x)).str.split().apply(
            lambda x: ' '.join(x)).apply(
            lambda u: re.sub(r'   ', r'', u)).apply(
            lambda v: re.sub(r'  ', r'', v))

        print('\n')
        print(i, '\n')
        bigram = album_lyrics.map(lambda x: find_ngrams(x.split(" "), 2))
        bigrams = bigram.tolist()
        bigrams = list(chain(*bigrams))
        bigrams = [(x.lower(), y.lower()) for x,y in bigrams if x != y]
        bigrams_counts = Counter(bigrams)
        print('\nBigrams\n')
        pp.pprint(bigrams_counts.most_common(n))
        
        trigram = album_lyrics.map(lambda x: find_ngrams(x.split(" "), 3))
        trigrams = trigram.tolist()
        trigrams = list(chain(*trigrams))
        trigrams = [(x.lower(), y.lower(), z.lower()) for x, y, z in trigrams if (x != y and y != z and x != z) and (x != z)]
        trigrams_counts = Counter(trigrams)
        print('\nTrigrams\n')
        pp.pprint(trigrams_counts.most_common(n))
        
        quadgram = album_lyrics.map(lambda x: find_ngrams(x.split(" "), 4))
        quadgrams = quadgram.tolist()
        quadgrams = list(chain(*quadgrams))
        quadgrams = [(x.lower(), y.lower(), z.lower(), a.lower()) for x, y, z, a in quadgrams if x != y]
        quadgrams_counts = Counter(quadgrams)
        print('\nQuadgrams\n')        
        pp.pprint(quadgrams_counts.most_common(n))
        

albumify(kanye, 10)
        
        
        
        




The College Dropout 


Bigrams

[   (('i', 'm'), 114),
    (('and', 'i'), 77),
    (('don', 't'), 69),
    (('ain', 't'), 68),
    (('it', 's'), 52),
    (('you', 'know'), 49),
    (('to', 'the'), 46),
    (('that', 's'), 44),
    (('in', 'the'), 44),
    (('kanye', 'west'), 42)]

Trigrams

[   (('are', 'you', 'gonna'), 30),
    (('you', 'gonna', 'be'), 30),
    (('i', 'ain', 't'), 28),
    (('all', 'these', 'things'), 26),
    (('gonna', 'be', 'are'), 24),
    (('be', 'are', 'you'), 24),
    (('get', 'up', 'i'), 23),
    (('up', 'i', 'get'), 23),
    (('things', 'all', 'these'), 22),
    (('these', 'things', 'all'), 20)]

Quadgrams

[   (('are', 'you', 'gonna', 'be'), 30),
    (('you', 'gonna', 'be', 'are'), 24),
    (('gonna', 'be', 'are', 'you'), 24),
    (('be', 'are', 'you', 'gonna'), 24),
    (('get', 'up', 'i', 'get'), 23),
    (('all', 'these', 'things', 'all'), 20),
    (('these', 'things', 'all', 'these'), 20),
    (('things', 'all', 'these', 'things'), 20),
    (('it', 'al

[   (('of', 'the', 'lord'), 23),
    (('til', 'the', 'power'), 22),
    (('sing', 'til', 'the'), 19),
    (('we', 'need', 'you'), 18),
    (('the', 'power', 'of'), 16),
    (('power', 'of', 'the'), 16),
    (('the', 'lord', 'comes'), 15),
    (('lord', 'comes', 'down'), 15),
    (('that', 's', 'on'), 12),
    (('we', 'have', 'everything'), 10)]

Quadgrams

[   (('sing', 'til', 'the', 'power'), 19),
    (('til', 'the', 'power', 'of'), 16),
    (('the', 'power', 'of', 'the'), 16),
    (('power', 'of', 'the', 'lord'), 16),
    (('of', 'the', 'lord', 'comes'), 15),
    (('the', 'lord', 'comes', 'down'), 15),
    (('we', 'have', 'everything', 'we'), 10),
    (('have', 'everything', 'we', 'need'), 10),
    (('down', 'sing', 'til', 'the'), 9),
    (('everything', 'we', 'need', 'ooh'), 8)]


# Direct Features

In [ ]:
#Preprocess
def nltk_preprocess(data):
    '''This function preprocesses a data frame, specifing a text_column, 
    and strips down the document to cleaned, individualized word tokens without
    stop words and other excessive parts of speech and eventually rejoins the remaining words.
    '''
    #Initializes stop words and new column creation
    stop = stopwords.words('english')
    
    
    #Initialize Lemmatizer object and final list of lemmatized words
    lemmatizer = WordNetLemmatizer()
    
    def get_wordnet_pos(treebank_tag):
        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            return None # for easy if-statement
        
    def lemmatized(word, tag):
        wntag = get_wordnet_pos(tag)
        if wntag is None:
            lemma = str(lemmatizer.lemmatize(word))
        else:
            lemma = str(lemmatizer.lemmatize(word, pos=wntag))
        return lemma

    #Song Specifics
    data = data.str.lower()
    data = data.apply(lambda x: re.sub(r'intro', r'', x)).apply(
        lambda y: re.sub(r'verse', r'', y)).apply(
        lambda z: re.sub(r'chorus', r'', z)).apply(
        lambda a: re.sub(r'produce', r'', a))    
    data = data.apply(lambda x: re.sub("[\[].*?[\]]", "", x))

    #Basic
    data = data.apply(
        lambda x: re.sub(r'\d+', r' ', x)).apply(
        lambda y: re.sub(r'\W+', r' ', y)).apply(
        lambda z: re.sub(r"_+",r" ",z))
    data = data.apply(word_tokenize)
    data = data.apply(lambda x: [item for item in x if item not in stop])
    data = data.apply(pos_tag)
    data = data.apply(lambda x: [lemmatized(word, tag) for (word, tag) in x])
    data = data.apply(lambda x: ' '.join(x))
    data = data.apply(lambda u: re.sub(r'   ', r'', u)).apply(lambda v: re.sub(r'  ', r'', v))

    return data


In [ ]:
kanye['processed_lyrics'] = nltk_preprocess(kanye['lyrics'])

In [ ]:
vectorizer = CountVectorizer()
cv = vectorizer.fit_transform(kanye['processed_lyrics'])
cv_freq = pd.DataFrame({'term': vectorizer.get_feature_names(), 'occurrences':np.asarray(cv.sum(axis=0)).ravel().tolist()})
cv_freq['frequency'] = cv_freq['occurrences']/cv_freq.shape[0]
cv_freq.sort_values('frequency', ascending = False).head(50)


In [ ]:
kanye.to_pickle('kanye_modeling.pkl')